In [18]:
from typing import Dict, List, Tuple, Union, Any, Optional

import os
import sys
import time
print("Python version", sys.version)

# Ignore pandas warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pathlib import Path

import pandas as pd
import numpy as np
import tensorflow as tf

from tqdm.autonotebook import tqdm

import ray
from ray import train, tune

from src.datasets import daocensus

%load_ext autoreload
%autoreload 2

tf.get_logger().setLevel('WARNING')

tf.config.list_physical_devices("GPU")
sys_details = tf.sysconfig.get_build_info()
cuda = sys_details.get("cuda_version", -1)
cudnn = sys_details.get("cudnn_version", -1)
print(cuda, cudnn)
print('Ray version:', ray.__version__)

Python version 3.9.18 (main, Oct 24 2023, 09:18:18) 
[GCC 11.4.0]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
11.8 8
Ray version: 2.7.1


In [19]:
# Others config
SEED: int = 57
RAY_RESULTS_PATH: Path = Path('~/ray_results').expanduser()

# Dataset config
DAO_NAME = 'Decentraland'

# Dataset splits config
SPLIT_FREQ = '1W' # Split weekly
LAST_SPLITS = 10 # Use just last 10 splits

# Training config
MAX_EPOCHS: int = 200
EPOCHS_PER_ITER: int = 5
SAMPLES_PER_SPLIT: int = 100
OPTIM_METRIC: str = 'ndcg@10'

# Eval config
TOP_K: List[int] = [5, 10]
METRICS: List[str] = ["recall", "ndcg", "precision", "map"]

# Load data

In [20]:
dfptext = pd.read_csv('./snapshot_proposals.csv')[['proposal_id', 'title', 'description', 'start', 'end']]
dfv, dfp = daocensus.get("./data/daos-census", DAO_NAME, 'snapshot')
dfv['voter'] = dfv['voter'].astype('str')
dfp = dfp.merge(dfptext, how='left', left_on='platform_proposal', right_on='proposal_id')
dfp[['start', 'end']] = dfp[['start', 'end']].astype('datetime64')
print(dfv.info())
print(dfp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       116560 non-null  object        
 1   name           116560 non-null  object        
 2   id             116560 non-null  object        
 3   proposal       116560 non-null  category      
 4   deployment     116560 non-null  object        
 5   platform_vote  116560 non-null  object        
 6   voter          116560 non-null  object        
 7   date           116560 non-null  datetime64[ns]
 8   choice         116560 non-null  object        
 9   weight         116560 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(7)
memory usage: 8.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1942 entries, 0 to 1941
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------ 

## Transform data

In [21]:
def to_microsoft(dfv):
    df = dfv[['voter', 'proposal', 'date']].rename(columns={
        'voter': 'userID',
        'proposal': 'itemID',
        'date': 'timestamp',
    })
    df['itemID'] = df['itemID'].astype('str')
    df['rating'] = 1
    return df

df = to_microsoft(dfv)
df.head()

,userID,itemID,timestamp,rating
0,0xe7af1c70f8f089c4c3bd71999692c6c5a15d9e2a,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 12:28:01,1
1,0xc54a6c3778016b06cbd126ccc3b5bc06c5f666fb,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 02:16:23,1
2,0xd82d005e8f8d5385db40ba23884a5c967bb1e8af,b86aa059-3d31-5d41-a472-70962816f779,2021-12-17 00:38:22,1
3,0xf4c64db66ffb301985f5ecd85c8f3f9c02f2659d,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:47:08,1
4,0xd5e9ef1cedad0d135d543d286a2c190b16cbb89e,b86aa059-3d31-5d41-a472-70962816f779,2021-12-16 18:32:15,1


# Split data

Each proposal remains open for a few days, our environment is different of a movies recommender system. For this reason, we will use a TimeSeriesSplit instead of a K-Fold to cross-validate the model.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)

El TimeSeriesSplit de scikit-learn no nos vale porque el número de elementos en cada split es el mismo, pero el tamaño del intervalo, no. Como queremos simular un comportamiento realista, haremos el split dividiendo por intervalos de igual longitud.

In [22]:
from recommenders.evaluation.python_evaluation import metrics as metrics_dict

In [23]:
from src.model_selection import timeFreqSplitCurrent, timeIntervalSplitCurrent

# max_train_prev = df['timestamp'].min().date()
# N_SPLITS = 10; SKIP_SPLIT = 0
# folds = list(timeIntervalSplitCurrent(df, N_SPLITS, dfp, skip=SKIP_SPLIT, remove_not_in_train_col='userID', return_open=True))
folds = list(timeFreqSplitCurrent(df, SPLIT_FREQ, dfp, return_open=True, remove_not_in_train_col='userID'))[-LAST_SPLITS:]
for i, (dftrain, dftest, t, open_proposals) in enumerate(folds):
    min_train = dftrain['timestamp'].min().date()
    max_train = dftrain['timestamp'].max().date()
    min_test  = dftest['timestamp'].min().date()
    max_test  = dftest['timestamp'].max().date()

    train_users = len(set(dftrain['userID']))
    test_users = len(set(dftest['userID']))
    
    print(f"Split {i}, train from: {min_train} to {max_train}, test from: {min_test} to {max_test}")
    print(f"  t: {t}")
    print(f"  len(train): {len(dftrain)}, len(test): {len(dftest)}")
    print(f"  users(train): {train_users}, users(test): {test_users}")

    print()
    dftest['prediction'] = 1
    for m in METRICS:
        f = metrics_dict[f'{m}_at_k']
        print(f"  highest possible {m}@{TOP_K[0]}:\t{f(dftest, dftest, k=TOP_K[0], relevancy_method='top_k'):.4f}")

    print("-"*30)

Split 0, train from: 2021-05-24 to 2023-05-07, test from: 2023-05-07 to 2023-05-14
  t: 2023-05-07 15:29:29
  len(train): 105603, len(test): 328
  users(train): 6846, users(test): 120

  highest possible recall@5:	0.9443
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.4267
  highest possible map@5:	0.9443
------------------------------
Split 1, train from: 2021-05-24 to 2023-05-14, test from: 2023-05-14 to 2023-05-21
  t: 2023-05-14 15:29:29
  len(train): 106687, len(test): 238
  users(train): 6871, users(test): 88

  highest possible recall@5:	0.9604
  highest possible ndcg@5:	1.0000
  highest possible precision@5:	0.4614
  highest possible map@5:	0.9604
------------------------------
Split 2, train from: 2021-05-24 to 2023-05-21, test from: 2023-05-21 to 2023-05-28
  t: 2023-05-21 15:29:29
  len(train): 108276, len(test): 450
  users(train): 6901, users(test): 144

  highest possible recall@5:	0.9309
  highest possible ndcg@5:	1.0000
  highest possible precision@5

# Defining training

In [24]:
# from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from src.models import LightGCNCustom
from recommenders.utils.python_utils import get_top_k_scored_items

## Small test of `LightGCNCustom`

In [25]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

In [26]:
hparams = prepare_hparams(
    model_type='lightgcn',
    n_layers=3,
    batch_size=512,
    embed_size=64,
    epochs=2,
    learning_rate=0.001,
    decay=0.001,
    metrics=["recall", "ndcg", "precision", "map"],
    eval_epoch=2,
    top_k=TOP_K[0],
    save_model=False,
    MODEL_DIR='./data/model/lightgcn/',
)
dataloader = ImplicitCF(train=folds[-1][0], test=folds[-1][1], seed=SEED)
print("items:", dataloader.n_items, "user:", dataloader.n_users)
model = LightGCNCustom(data=dataloader, hparams=hparams)

items: 1939 user: 7206
Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [27]:
model.fit()

Epoch 1 (train)0.7s: train loss = 0.59212 = (mf)0.59128 + (embed)0.00084
Epoch 2 (train)0.6s + (eval)0.1s: train loss = 0.34023 = (mf)0.33596 + (embed)0.00428, recall = 0.00938, ndcg = 0.00913, precision = 0.00360, map = 0.00688


In [28]:
model.run_eval()

[0.009384384384384385,
 0.009129748836662984,
 0.0036036036036036037,
 0.006881881881881881]

In [29]:
model.recommend_k_items(dataloader.test, top_k=3, use_id=True, remove_seen=True, recommend_from={'b86aa059-3d31-5d41-a472-70962816f779', '56b4d333-4138-5aa3-9890-3502b9478079', 'd083109e-4819-54b9-a01c-67bd5a770f65' })

,userID,itemID,prediction
0,397,0,-9.397598
3,5795,1938,0.454089
4,5795,0,-1.289321
6,328,0,-5.120289
9,266,0,-10.686196
...,...,...,...
661,1292,1938,-0.659605
662,1292,1238,-2.836244
663,1293,0,1.205570
664,1293,1938,-0.508326


# Defining trainer

In [30]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

class TrainLightGCN(tune.Trainable):
    def setup(
        self,
        config: Dict[str, Any],
        data,
    ):
        self.config = config

        self.hparams = prepare_hparams(
            model_type='lightgcn',
            n_layers=config['conv_layers'],
            batch_size=2**config['batch_size'],
            embed_size=config['embedding_dim'],
            epochs=EPOCHS_PER_ITER,
            learning_rate=config['learning_rate'],
            decay=config['l2'],
            metrics=METRICS,
            eval_epoch=-1,
            top_k=TOP_K[0],
            save_model=False,
            MODEL_DIR='./data/model/lightgcn/',
        )

        train, test, self.t, self.open_proposals = data
        self.dataloader = ImplicitCF(train=train, test=test, seed=SEED)
        self.model = LightGCNCustom(self.hparams, self.dataloader, seed=SEED)
        self.total_train = 0
        self.total_eval = 0

    @property
    def iteration(self):
        return self.model.epochs_done

    @property
    def training_iteration(self):
        return self.model.epochs_done

    def step(self):
        """
        As a rule of thumb, the execution time of step should be large enough to avoid overheads 
        (i.e. more than a few seconds), but short enough to report progress periodically 
        (i.e. at most a few minutes).
        """
        assert EPOCHS_PER_ITER > 0

        train_start = time.time()
        for _ in range(EPOCHS_PER_ITER):
            ret = self.model.fit_epoch()
        eval_start = train_end = time.time()

        eval_dict = {'model_'+k:v for k,v in zip(self.model.metrics, self.model.run_eval())}
        for k in TOP_K:
            recs = self.model.recommend_k_items(
                self.dataloader.test, 
                top_k=k,
                use_id=True, 
                remove_seen=True, 
                recommend_from=self.open_proposals,
            )
            
            eval_dict[f'precision@{k}'] = precision_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'ndcg@{k}'] = ndcg_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'recall@{k}'] = recall_at_k(self.dataloader.test, recs, k=k)
            eval_dict[f'map@{k}'] = map_at_k(self.dataloader.test, recs, k=k)

        eval_end = time.time()

        self.total_train += train_end - train_start
        self.total_eval += eval_end - eval_start
        
        return {
            'iteration': self.iteration,
            'loss': ret[0],
            'mf_loss': ret[1],
            'emb_loss': ret[2],
            **eval_dict,
            'time_train': train_end-train_start,
            'time_test': eval_end-eval_start,
            'time_total_train': self.total_train,
            'time_total_test': self.total_eval,
        }

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model")
        self.model.saver.save(
            sess=self.model.sess,
            save_path=checkpoint_path,
        )
        return checkpoint_dir

    def load_checkpoint(self, checkpoint_path):
        self.model.load(checkpoint_path)

# Big experiment

In [31]:
os.uname().nodename

'lamarck'

In [32]:
RAY_RESULTS_PATH

PosixPath('/home/daviddavo/ray_results')

In [33]:
import ray
from ray.tune.search.hyperopt import HyperOptSearch
import datetime as dt

def getTunerOnFold(f, points_to_evaluate = None):
    name = f'LightGCN_dao={DAO_NAME},freq={SPLIT_FREQ},fold={f}'
    paths = list(RAY_RESULTS_PATH.glob(f'{name}_*'))
    last_experiment = max(paths, key=lambda x: x.stat().st_ctime) if paths else None
    
    ### SET TRAINING RESOURCES
    if os.uname().nodename == 'lamarck':
        # assert torch.cuda.is_available()
        
        NUM_SAMPLES = SAMPLES_PER_SPLIT
        # Every run takes approx half a gig of vram (no optimizations)
        # The RTX 4090 has 24GB so we can run the model about 48 times
        resources_per_trial={
            'cpu': 1,
            # GPU has 25GiB, and each run might take up to 2GiB (torch version was lighter)
            # so each run might take up to 1/12th of the GPU
            # I use 1/16th so I don't take all the resources in the machine
            'gpu': 1/16,
        }
    else:
        NUM_SAMPLES = 1
        resources_per_trial={
            'cpu': 1,
            # It takes about 1.5 GiB with full training data, but I put a bit more because
            # this notebook also takes a bit of memory
            'memory': 2e9,
        }

    dftrain,dftest,t,open_proposals = folds[f]
    param_space = dict(
        fold=f,
        batch_size=tune.randint(6,10), # 64 - 1024
        embedding_dim=tune.lograndint(1, 1024, base=2),
        conv_layers=tune.randint(1,6),
        learning_rate=tune.qloguniform(1e-4, 1, 1e-4),
        l2=tune.loguniform(1e-7, 1e-2),
    )
    
    ### RESTORE EXPERIMENT OR CREATE A NEW ONE
    if last_experiment and tune.Tuner.can_restore(last_experiment):
        print(f"Restoring last experiment: {last_experiment}")
        tuner = tune.Tuner.restore(
            str(last_experiment),
            trainable=tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds[f]),
                resources_per_trial,
            ),
            restart_errored=True,
            param_space=param_space,
        )
    else:
        print(f"No experiment found for fold {f}, creating new tuner with {NUM_SAMPLES} samples")
        search_alg = None
        search_alg = HyperOptSearch(
            # points_to_evaluate=[{
            #     'batch_size': 8, # 2**8 = 256
            #     'learning_rate': 10e-2,
            #     'l2': 10e-6,
            #     'embedding_dim': 100,
            #     'conv_layers': 3,
            # }],
            points_to_evaluate = points_to_evaluate,
            random_state_seed=SEED,
        )
        # search_alg = tune.search.Repeater(search_alg, N_SPLITS-SKIP_SPLIT)
        
        tuner = tune.Tuner(
            tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds[f]),
                resources_per_trial,
            ),
            run_config=train.RunConfig(
                stop={'training_iteration': MAX_EPOCHS/EPOCHS_PER_ITER, 'time_total_train': 300},
                name=name + f'_{dt.datetime.now().isoformat()}',
                storage_path=RAY_RESULTS_PATH,
            ),
            param_space=param_space,
            tune_config=tune.TuneConfig(
                search_alg=search_alg,
                num_samples=NUM_SAMPLES,
                metric=OPTIM_METRIC,
                mode='max',
            )
        )

    return tuner

In [36]:
tuners = []
results = []
last_best_result = None
for f in range(LAST_SPLITS):
    best_prev_config = None
    if last_best_result is not None:
        best_prev_config = last_best_result.config.copy()
        best_prev_config['fold'] += 1
        best_prev_config = [best_prev_config]
    
    t = getTunerOnFold(f, best_prev_config)
    tuners.append(t)

    rg = t.fit()
    results.append(rg)

    # Assert that the prev config has been tried
    if last_best_result is not None:
        if not any( 
            all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold'))
            for r in rg if r.config
        ):
            print("Best config:", last_best_result.config)
            assert False, f"The best config has not been tested in fold {f}"    
    
    last_best_result = rg.get_best_result()

2023-11-30 16:08:45,370	INFO experiment_state.py:529 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2023-11-30 16:08:45,372	WARNING tune_controller.py:516 -- Attempting to resume experiment from /home/daviddavo/ray_results/LightGCN_dao=Decentraland,freq=1W,fold=5_2023-11-28T15:07:45.041321. This will ignore any new changes to the specification.
2023-11-30 16:08:45,372	INFO tune_controller.py:520 -- Using the newest experiment state file found within the experiment directory: experiment_state-2023-11-30_16-08-02.json
2023-11-30 16:08:45,877	INFO tune.py:1143 -- Total run time: 0.52 seconds (0.00 seconds for the tuning loop).


Best config: {'fold': 4, 'batch_size': 7, 'embedding_dim': 0, 'conv_layers': 3, 'learning_rate': 0.5703, 'l2': 0.002807856907433476}


AssertionError: The best config has not been tested in fold 5